In [1]:
!pip install scikit-surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357280 sha256=0f70282ad44c031c76a6d0fc6547a3f0d943c97dd53959c0fb95db87d48015cb
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [2]:
import pandas as pd
from surprise import Dataset, Reader
from surprise import KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy
import os
import kagglehub
import pandas as pd

In [6]:
path = kagglehub.dataset_download("saurav9786/amazon-product-reviews")
print("Path to dataset files:", path)

100%|██████████| 109M/109M [00:00<00:00, 143MB/s] 

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/saurav9786/amazon-product-reviews/versions/1


In [7]:


dataset_path = '/root/.cache/kagglehub/datasets/saurav9786/amazon-product-reviews'
if os.path.exists(dataset_path):
    print("Files in the dataset folder:", os.listdir(dataset_path))
else:
    print(f"Directory does not exist: {dataset_path}")

Files in the dataset folder: ['versions', '1.complete']


In [8]:
file_path = '/root/.cache/kagglehub/datasets/saurav9786/amazon-product-reviews/versions/1/ratings_Electronics (1).csv'
data = pd.read_csv(file_path)

In [9]:
data = pd.read_csv(file_path, names=["userId", "productId", "Rating", "timestamp"], header=0)

In [10]:
data = data[["userId", "productId", "Rating"]]

In [11]:
data = data.iloc[:100000]

In [12]:
reader = Reader(rating_scale=(1, 5))
dataset = Dataset.load_from_df(data, reader)

In [13]:
trainset, testset = train_test_split(dataset, test_size=0.25, random_state=42)

User Based System

In [14]:
print("User-Based Collaborative Filtering:")
sim_options_user = {
    "name": "cosine",
    "user_based": True,
}
model_user = KNNBasic(sim_options=sim_options_user)
model_user.fit(trainset)
predictions_user = model_user.test(testset)
print("User-Based RMSE:", accuracy.rmse(predictions_user))


User-Based Collaborative Filtering:
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.3504
User-Based RMSE: 1.3503507780248063


Item Based System

In [15]:
print("\nItem-Based Collaborative Filtering:")
sim_options_item = {
    "name": "cosine",
    "user_based": False,
}
model_item = KNNBasic(sim_options=sim_options_item)
model_item.fit(trainset)
predictions_item = model_item.test(testset)
print("Item-Based RMSE:", accuracy.rmse(predictions_item))



Item-Based Collaborative Filtering:
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.3520
Item-Based RMSE: 1.3520010137832479


In [16]:
def get_recommendations(model, user_id, n=5):
    user_ratings = data[data["userId"] == user_id]["productId"].tolist()
    all_products = data["productId"].unique()
    products_to_predict = [prod for prod in all_products if prod not in user_ratings]

    predictions = [model.predict(user_id, prod) for prod in products_to_predict]
    predictions.sort(key=lambda x: x.est, reverse=True)
    return predictions[:n]


In [17]:
user_id = data["userId"].iloc[0]
recommendations = get_recommendations(model_user, user_id)
print(f"\nTop 5 Recommendations for User {user_id}:")
for rec in recommendations:
    print(f"Product ID: {rec.iid}, Predicted Rating: {rec.est:.2f}")


Top 5 Recommendations for User A2CX7LUOHB2NDG:
Product ID: 0439886341, Predicted Rating: 4.03
Product ID: 0511189877, Predicted Rating: 4.03
Product ID: 0528881469, Predicted Rating: 4.03
Product ID: 0558835155, Predicted Rating: 4.03
Product ID: 059400232X, Predicted Rating: 4.03
